# Extracting and processing STM Image Data from Omicron's propritary .Z_MTRX format 
In order to extract the images, first import the necessary packages. We will be using NanoscoPy to read the STM data and matplotlib to show the data.

In [4]:
import sys
import matplotlib.pyplot as plt
from pathlib import Path

In [3]:
# Custom modules
sys.path.append('../../NanoscoPy/')
from nanoscopy import spm
from nanoscopy.utilities import dialog_askdirectory 

### Get Root Folder Path
Prompt the user for the directory which contains the .Z_mtrx files and the result file chain.

In [5]:
folder_path = dialog_askdirectory()

### Get Root Folder Path
Prompt the user for the directory which contains the .Z_mtrx files and the result file chain.

In [4]:
sample_batch_name = Path(folder_path).parts[-1]

### Read all STM files in the folder

In [6]:
scans = spm.read(folder_path)

Reading 4 sxm files...DONE
Processing 1 sxm files...Found the following mtrx files in path:
Processing WSe2--1_1...DONE


In [7]:
scans[0].as_dataframe()

,channels,paths,setpoints,voltages,widths,heights,datetimes,directions,traces,images
0,Z,G:/Research/AI_STM/NanoscoPy/ExampleDataFiles\...,1.000000e-09,1.000000e-09,1.000000e-09,1.000000e-09,2021-02-25T11:36:19,down,forward,"[[-1.204225128503822e-07, -1.204216886208087e-..."
1,Z,G:/Research/AI_STM/NanoscoPy/ExampleDataFiles\...,1.000000e-09,1.000000e-09,1.000000e-09,1.000000e-09,2021-02-25T11:36:19,down,backward,"[[-1.204103625696007e-07, -1.2040902674925746e..."
0,Current,G:/Research/AI_STM/NanoscoPy/ExampleDataFiles\...,1.000000e-09,1.000000e-09,1.000000e-09,1.000000e-09,2021-02-25T11:36:19,down,forward,"[[1.0036380615474627e-09, 9.974532311218809e-1..."
1,Current,G:/Research/AI_STM/NanoscoPy/ExampleDataFiles\...,1.000000e-09,1.000000e-09,1.000000e-09,1.000000e-09,2021-02-25T11:36:19,down,backward,"[[1.0091439905934863e-09, 1.007642969064193e-0..."


### Correct the forward sweep data
Each image is globally flattened via plane correction, followed by 2nd order polynomial background subtraction, line-by-line offset flattening, 3x3 gaussian smoothing, and CLAHE.

In [ ]:
corrected_scans = spm.correct([mtrx_file.data['Z'][0] for mtrx_file in scans])

### Show a comparison plot between some of the raw and processed images

In [ ]:
n = 6
fig, axs = plt.subplots(n, 2)
fig.suptitle(f'Sample Date: {sample_batch_name}')
fig.set_figheight(15)
fig.set_figwidth(7)
for i in range(n):
    raw = scans[i]
    processed = corrected_scans[i]
    sample_name = f'{raw.filename[-16:]}'

    axs[i, 0].imshow(raw.data['Z'][0] , origin = 'lower')
    axs[i, 0].set_title(sample_name)
    axs[i, 1].imshow(processed , origin = 'lower')
    axs[i, 1].set_title(f'post-processed {sample_name}')

# Hide x labels and tick labels for top plots and y ticks for right plots.
for ax in axs.flat:
    ax.xaxis.set_visible(False)
    ax.yaxis.set_visible(False)

### Output directories 

In [ ]:
dst_paths = [f'{Path(scan.path).parent}\\{scan.summary()}.jpg' for scan in scans]
dst_paths[:4]

### Export the data as jpg

In [ ]:
spm.io.export_spm(corrected_scans, dst_paths)